<a href="https://colab.research.google.com/github/eip4-mars/EIP4P2/blob/master/Session2/Initial_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')

model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 4)
model = model.to(device)
summary(model, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:
!ls -l drive/My\ Drive/Session\ 2\ Dataset 

ls: 'drive/My Drive/Session 2 Dataset': No such file or directory
lrw------- 1 root root 0 Jul 23 14:20 'drive/My Drive/Session 2 Dataset' -> '/content/drive/.shortcut-targets-by-id/1gsDgSBaoqrM6-W1b9HXuMRZc9SzwM98d/Session 2 Dataset'


In [36]:
cp -r /content/drive/.shortcut-targets-by-id/1gsDgSBaoqrM6-W1b9HXuMRZc9SzwM98d/Session\ 2\ Dataset Session2_Dataset

cp: cannot stat '/content/drive/.shortcut-targets-by-id/1gsDgSBaoqrM6-W1b9HXuMRZc9SzwM98d/Session 2 Dataset/FlyBirds Canada': No such file or directory


In [59]:
%%timeit
!du -sh Session2_Dataset/

4.4G	Session2_Dataset/
4.4G	Session2_Dataset/
4.4G	Session2_Dataset/
4.4G	Session2_Dataset/
1 loop, best of 3: 1.73 s per loop


In [50]:
!ls -Slhr Session2_Dataset/Flying\ Birds | tail -500 |head -1 

-rw------- 1 root root  1.1M Jul 23 15:40 gz_all-free-download.com1.jpg


In [47]:
!ls -Slhr Session2_Dataset/Large\ QuadCopters | tail -200 |head -1 

-rw------- 1 root root  473K Jul 23 14:51 74098-ethos-flight.jpg


In [60]:
!ls -Slhr Session2_Dataset/Small\ QuadCopters | tail -200 |head -1 

-rw------- 1 root root  90K Jul 23 15:58 abstract-conceptual-child-playing-drone-remote-sensing-la-felicità-di-un-bambino-con-giocattolo-moderno-e-pericoloso-107771953.jpg


In [49]:
!ls -Slhr Session2_Dataset/Winged\ Drones | tail -200 |head -1 

-rw------- 1 root root  672K Jul 23 15:18 wing-testers-receiving-package-as-part-of-faa-testing_wide-555d7c5d81e7172b02979c27f18b25585106abbd.jpg


In [62]:
!ls -la /content/drive

total 12
drwx------ 24 root root 4096 Jul 23 14:38 'My Drive'
dr-x------  3 root root 4096 Jul 23 14:38  .shortcut-targets-by-id
drwx------  2 root root 4096 Jul 23 14:38  .Trash


In [20]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

path = "/content/Session2_Dataset/"
classes = os.listdir(path)
train = []
test = []
labels = pd.DataFrame(classes,index=range(len(classes)), columns=['Label'])
labels.reset_index(inplace=True)


test_ratio = 0.2

#print(labels)
for i in range(len(classes)):
    child_path = path+classes[i]+'/'
    imgs = os.listdir(child_path)
    #print(len(tr),classes[i])
    trn,tst = train_test_split(imgs, test_size=test_ratio, random_state=42)
    
    train += [[child_path+x, i] for x in trn]
    test += [[child_path+x, i] for x in tst]
    
train_df = pd.DataFrame(train,columns=['File_name', 'label_index'])
test_df = pd.DataFrame(test,columns=['File_name', 'label_index'])

print(train_df.shape, test_df.shape, labels.shape)
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)
labels.to_csv('labels.csv',index=False)

(17443, 2) (4362, 2) (4, 2)


In [28]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import random

class Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        self.frame = pd.read_csv(csv_file, header=0)
        self.transform = transform

    def __getitem__(self, idx):
        image_name = self.frame.iloc[idx, 0]
        label = self.frame.iloc[idx, 1]

        image = Image.open(image_name)
        

        sample = {'image': image_name, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return len(self.frame)

def getTrainData(batch_size=64):
    
    stats = {'mean': [0.3931, 0.3785, 0.3606],
             'std': [0.1965, 0.1813, 0.1779]}

    transformed_training = Dataset(csv_file='/content/train.csv',
                                        transform=transforms.Compose([
                                            Resize(224),
                                            CenterCrop(224),
                                            RandomHorizontalFlip(),
                                            RandomRotate(5),
                                            ToTensor(),
                                            Normalize(stats['mean'],
                                                      stats['std'])
                                        ]))

    dataloader_training = DataLoader(transformed_training, batch_size,
                                     shuffle=True, num_workers=4, pin_memory=False)

    return dataloader_training



def getTestData(batch_size=64):

    stats = {'mean': [0.3931, 0.3785, 0.3606],
             'std': [0.1965, 0.1813, 0.1779]}
    
    transformed_testing = depthDataset(csv_file='/content/test.csv',
                                       transform=transforms.Compose([
                                           Resize(224),
                                            CenterCrop(224),
                                           ToTensor(is_test=True),
                                           Normalize(stats['mean'],
                                                     stats['std'])
                                       ]))

    dataloader_testing = DataLoader(transformed_testing, batch_size,
                                    shuffle=False, num_workers=4, pin_memory=False)

    return dataloader_testing


In [25]:
train_loader = getTrainingData(64)
test_loader = getTestData(64)


,File_name,label_index
0,/content/Session2_Dataset/Large QuadCopters/ph...,0
1,/content/Session2_Dataset/Large QuadCopters/1l...,0
2,/content/Session2_Dataset/Large QuadCopters/02...,0
3,/content/Session2_Dataset/Large QuadCopters/ge...,0
4,/content/Session2_Dataset/Large QuadCopters/51...,0
